<a href="https://colab.research.google.com/github/aecins/tutorials/blob/main/least_squares/ordinary_least_squares.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import scipy.linalg

## Create a linear least squares problem ([wiki](https://en.wikipedia.org/wiki/Linear_regression))
We setup a simple linear least squares problem by creating values for:
- true parameter values $x$ (sampled from uniform distribution)
- independent variables $A$ (sampled from uniform distribution)
- dependent variables $b$ (constructed as $Ax + \mathcal{N}(0, I * \sigma_b)$, i.e. residuals are uncorrelated and have the same variance)

This satisfies the assumptions on the measurement errors of ordinary least squares:
- Measurement errors are uncorrelated
- Measurement errors have the same variance (this property is called [homoscedasticity](https://en.wikipedia.org/wiki/Homoscedasticity))
- Measurement errors are zero mean normally distributed

These assumptions can be summarized as a single assumption:
- measurement error vector is drawn from a multivariate Gaussian distribution with zero mean and a covariance matrix of $I * \sigma_b^2$.

In [2]:
# Function that generates random values drawn from uniform distribution with a given range.
def generate_random_uniform(y_size, x_size, value_min, value_max):
    assert(value_max > value_min)

    values = np.random.rand(y_size, x_size)
    values = values * (value_max - value_min)
    values = values + value_min

    return values

In [3]:
# Create true parameters of a linear least squares problem by sampling them uniformly
# in the range [-X_TRUE_MAGNITUDE; X_TRUE_MAGNITUDE]
NUM_PARAMETERS = 3
X_TRUE_MAGNITUDE = 10

x_true = generate_random_uniform(NUM_PARAMETERS, 1, -X_TRUE_MAGNITUDE, X_TRUE_MAGNITUDE)
print(x_true)

[[ 7.23110774]
 [-2.4345187 ]
 [ 6.18771675]]


In [4]:
# Function that generates random independent variables (A) and corresponding noisy dependent variables (b)
# for a linear least squares problem.
def generate_Ab(x_true, num_measurements, A_magnitude, b_noise_sigma):
    # First generate independent variables aka A matrix.
    # These are generated from a uniform distribution in the range [-A_magnitude; A_magnitude]
    A = generate_random_uniform(NUM_MEASUREMENTS, NUM_PARAMETERS, -A_magnitude, A_magnitude)

    # Next generate dependent variables aka vector b.
    # These are generated as values predicted by the true values of the model + measurement noise
    measurement_noise = (np.random.normal(0, b_noise_sigma, [num_measurements, 1]))
    b = A.dot(x_true) + measurement_noise

    return [A, b]

In [5]:
# Generate measurements.
NUM_MEASUREMENTS = 100000
A_MAGNITUDE = 0.1
B_NOISE_SIGMA = 10

[A, b] = generate_Ab(x_true, NUM_MEASUREMENTS, A_MAGNITUDE, B_NOISE_SIGMA)

## Validate residuals
The vector of residuals is defined as:
$$
r = Ax - b
$$
We can check that the system was constructed correctly by checking that residuals evaluated at ground truth parameter values have the same distribution as the noise that was added to dependent measurements $b$:
$$
b = Ax_{true} + \mathcal{N}(0, \sigma_b) \\
r = Ax_{true} - b \\
r \sim \mathcal{N}(0, \sigma_b)
$$

In [6]:
# Check that the residuals evaluated at true parameter values have the same statistics as measurement noise.
residuals_true = A.dot(x_true) - b
print("Statistics of residuals evaluated at true parameter values")
print("mean  : estimated {:.3f}, expected {:.3f}".format(np.mean(residuals_true), 0))
print("sigma : estimated {:.3f}, expected {:.3f}".format(np.std(residuals_true), B_NOISE_SIGMA))

Statistics of residuals evaluated at true parameter values
mean  : estimated -0.048, expected 0.000
sigma : estimated 9.989, expected 10.000


## Jacobian
[Jacobian](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant) of a vector-valued function is defined as a matrix of partial derivatives of the value of the function with respect to its parameters . In the context of optimization we are interested in the Jacobian of the residuals since the tells us how the residuals change as we change the parameters.

For a linear least squares problem we have:
$$
\begin{align*}
r(x) & = Ax - b \\
J & = \begin{bmatrix}\dfrac{dr_i}{dx_j}\end{bmatrix} = A
\end{align*}
$$

### Validating Jacobian
For any linear function $f(x)$ with Jacobian $J$ we can predict the value of a function at one point given the value of the function at another point:
$$
f(x) = f(y) + J * (x - y) = f(y) + J \Delta
$$
where $\Delta = x-y$ is a vector and $J \Delta$ is a matrix vector product.

In [7]:
# Validate Jacobian of the residual function.
NUM_JACOBIAN_TESTS = 100
max_value_difference = 0

# Calculate Jacobian.
J = A

for i in range(NUM_JACOBIAN_TESTS):
    # Create two random parameter values.
    x = generate_random_uniform(NUM_PARAMETERS, 1, -X_TRUE_MAGNITUDE, X_TRUE_MAGNITUDE)
    y = generate_random_uniform(NUM_PARAMETERS, 1, -X_TRUE_MAGNITUDE, X_TRUE_MAGNITUDE)

    # Calculate residuals
    r_x = A.dot(x) - b
    r_y = A.dot(y) - b

    # Calculate predicted residual
    delta = x - y
    r_x_predicted = r_y + J.dot(delta)
    max_value_difference = max(max_value_difference, np.linalg.norm(r_x - r_x_predicted))

print("Maximum norm of the difference between true and predicted values of the residual: {:.2f}".format(max_value_difference))

Maximum norm of the difference between true and predicted values of the residual: 0.00


## Normal equations
An optimal solution $x^*$ to the linear least squares problem is the solution to the system of linear equations called the [normal equations](https://en.wikipedia.org/wiki/Ordinary_least_squares#Matrix/vector_formulation):
$$A^TA x^* = A^Tb$$
TODO: add link to normal equations derivation

### Information matrix / Hessian
$A^TA$ is also known as the information matrix $\mathcal{I}$. It can also be interpreted as the matrix of second order partial derivatives of the least squares cost function with respect to parameters $x$. In other words $A^TA$ is the [Hessian](https://en.wikipedia.org/wiki/Hessian_matrix) of the cost function:
$$
f(x) = ||Ax - b||^2 \\
H = \begin{bmatrix} \dfrac{\partial f}{\partial x_i  \partial x_j} \end{bmatrix} = A^TA
$$



## Solving by inverting the information matrix
Optimal solution to the problem can be found by inverting the information matrix:

$$x^* = (A^T A)^{-1} A^T b$$

In [8]:
# Solve linear least squares problem by inverting the information matrix.
def solve_using_matrix_inverse(A, b):
    I = A.transpose().dot(A)
    I_inv = np.linalg.inv(I)
    Atb = (A.transpose()).dot(b)
    return np.matmul(I_inv, Atb)

%time x_estimated = solve_using_matrix_inverse(A, b)

CPU times: user 1.9 ms, sys: 1.96 ms, total: 3.86 ms
Wall time: 1.94 ms


In [9]:
# Calculate RMSE of parameters.
def parameter_rmse(x_estimated, x_true):
    assert(len(x_estimated) == len(x_true))
    return np.linalg.norm(x_estimated - x_true) / len(x_estimated)

rmse = parameter_rmse(x_estimated, x_true)
print("Root Mean Squared Error of paramters estimated using information matrix inverse:\n {:f}".format(rmse))

Root Mean Squared Error of paramters estimated using information matrix inverse:
 0.395545


## Solve using factorization of the information matrix
Using least squares problems by inverting the information matrix has several downsides:
- inverting large matrices is slow
- inverting matrices is numerically unstable

Alternatively, we can take advantage of the special structure of the information matrix. Since $\mathcal{I} = A^TA$ is a real summetric matrix we can factorize it using the [LDL decomposition](https://en.wikipedia.org/wiki/Cholesky_decomposition#LDL_decomposition):
$$A^TA = LDL^T$$
where $L$ is a lower diagonal matrix with unit diagonal and $D$ is a diagonal matrix. Normal equations become:
$$
LDL^Tx^* = A^Tb
$$
The solution can be obtained by solving a series of [systems of linear equations](https://en.wikipedia.org/wiki/System_of_linear_equations) (not to be confused with linear least squares problems!):
1. $Ly_1 = A^Tb$ (solved using [forward substituition](https://en.wikipedia.org/wiki/Triangular_matrix#Forward_and_back_substitution) since $L$ is lower triangular)
1. $Dy_2 = y_1$ (solved by element-wise division since $D$ is a diagonal matrix)
1. $L^Tx^* = y_2$ (solved using [back substituition](https://en.wikipedia.org/wiki/Triangular_matrix#Forward_and_back_substitution) since $L^T$ is upper triangular)

Note that none of the steps in the solution (calculating $A^TA$ and $A^Tb$, computing the LDL factorization of $A^TA$) and solving the resulting linear system of equations) require expensive operations like taking matrix inverse, squaring or taking square roots. In fact they only requires multiply-adds. This makes this approach both fast and numerically stable.

$LDL^T$ solver is implemeted in the [Eigen C++ library](https://eigen.tuxfamily.org/dox/classEigen_1_1LDLT.html#aa257dd7a8acf8b347d5a22a13d6ca3e1).

In [10]:
# Solve linear least squares problem using LDL factorization.
def solve_using_ldl(A, b):
    # Calculate LDL factorization of the information matrix
    I = A.transpose().dot(A)
    [L, D, perm] = scipy.linalg.ldl(I)

    # Solve Ly_1 = A^Tb
    y1 = scipy.linalg.solve_triangular(L, A.transpose().dot(b), trans=0, lower=True, unit_diagonal=True)

    # Solve Dy2 = y1
    diagonal = np.diagonal(D).reshape(-1, 1)
    y2 = np.divide(y1, diagonal)

    # Solve L^Tx^* = y_2
    return scipy.linalg.solve_triangular(L.transpose(), y2, trans=0, lower=False, unit_diagonal=True)

%time x_estimated = solve_using_ldl(A, b)
rmse = parameter_rmse(x_estimated, x_true)
print("Root Mean Squared Error of paramters estimated using LDL factorization:\n {:f}".format(rmse))

CPU times: user 1.8 ms, sys: 3.34 ms, total: 5.14 ms
Wall time: 2.58 ms
Root Mean Squared Error of paramters estimated using LDL factorization:
 0.395545


#### Note on the runtime
The algorithm used for calcularing matrix inverse in `numpy.linalg.inv` is optimized for performance. Hence we don't see a large improvement in the runtime between inverse information matrix and LDLT solvers.

## Solution covariance
### Estimating covariance using information matrix
Given a linear least squares problem $Ax = b$ the covariance matrix of the solution can be estimated as:
$$cov(x^*) = (A^T A)^{-1} \sigma_b^2$$
where $\sigma_b$ is the standard deviation of the residuals [[proof](https://github.com/aecins/tutorials/blob/main/least_squares/least_squares_covariance_derivation.ipynb)].

We expect the covariance matrix to be approximately equal to $I * \alpha$ for some scaling factor $\alpha$. This is because all of the measurements in $A$ and $b$ are independent (hence cross-covariance terms should be zero).

In [11]:
# Calculate the covariance matrix of the estimated parameters.
x_estimated_covariance = np.linalg.inv(np.matmul(A.transpose(), A)) * B_NOISE_SIGMA * B_NOISE_SIGMA
print("Solution covariance estimated using A matrix from a single problem:")
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
print(x_estimated_covariance)

Solution covariance estimated using A matrix from a single problem:
[[ 0.300  0.002  0.001]
 [ 0.002  0.299  0.001]
 [ 0.001  0.001  0.301]]


### Estimating covariane using bootstrap
Another way to estimate covariance of the solution is to use a bootstrap method. The high-level idea of this method is to:
1. generate multiple optimization problems that are "equivalent" to the original problem
2. solve these problems and get multiple estimates of the solution $x$
3. calculate the covariance matrix of the obtained solutionsand then calculate

Multiple ways were proposed to generate "equivalent" optimization problems ([wiki](https://en.wikipedia.org/wiki/Bootstrapping_(statistics))). One way is to resample with replacement the constraints in the optimization problems. That corresponds to resampling the rows in the matrix $A$ and corresponding values in $b$. In our case, since we are using synthetic data that is generated from a known distribution - we can simply generate multiple problems that:
- have the same solution $x$
- values in $A$ and $b$ are drawn from the same distribution

In [24]:
%%time

from tqdm.notebook import tqdm

# Calculate the covariance matrix using bootstrap.
# We create multiple problems with the same true solution but different measurements (A, b)(same measurement noise).
# We then calculate the covariance matrix of the solutions to these problems.
NUM_BOOTSTRAP_ITERATIONS = 10000
x_estimates = np.zeros((NUM_PARAMETERS, NUM_BOOTSTRAP_ITERATIONS))
for i in tqdm(range(NUM_BOOTSTRAP_ITERATIONS)):
    # Generate A and b.
    [A, b] = generate_Ab(x_true, NUM_MEASUREMENTS, A_MAGNITUDE, B_NOISE_SIGMA)

    # Solve.
    x_estimated = solve_using_matrix_inverse(A, b)

    # Append to solutions
    x_estimates[:, i] = x_estimated[:, 0]

print("Solution covariance estimated using bootstrap:")
print(np.cov(x_estimates))

  0%|          | 0/10000 [00:00<?, ?it/s]

Solution covariance estimated using bootstrap:
[[ 0.308 -0.004  0.002]
 [-0.004  0.300  0.003]
 [ 0.002  0.003  0.298]]
CPU times: user 2min 37s, sys: 2min 5s, total: 4min 43s
Wall time: 2min 53s


In [ ]:
# TODO: add an analytical derivation of the covariance of the solution.